In [70]:
import numpy as np
import torch
file_path = '/Users/tingxi/Downloads/sample_file/q_map_4.txt'
matrix = np.loadtxt(file_path)
matrix = matrix.reshape(1,1,128,128)
matrix = torch.tensor(matrix)
matrix

tensor([[[[0.1686, 0.1788, 0.1710,  ..., 0.1565, 0.1597, 0.1490],
          [0.1740, 0.1769, 0.1510,  ..., 0.1385, 0.1550, 0.1596],
          [0.1654, 0.1486, 0.1174,  ..., 0.1165, 0.1385, 0.1562],
          ...,
          [0.1588, 0.1477, 0.1199,  ..., 0.1154, 0.1394, 0.1574],
          [0.1637, 0.1667, 0.1476,  ..., 0.1390, 0.1572, 0.1614],
          [0.1487, 0.1638, 0.1587,  ..., 0.1575, 0.1616, 0.1512]]]],
       dtype=torch.float64)

In [71]:
def argmax2d(tensor):
  n = tensor.size(0)
  d = tensor.size(2)
  m = tensor.view(n, -1).argmax(1)
  return torch.cat(((m / d).view(-1, 1), (m % d).view(-1, 1)), dim=1)

argmax2d(matrix)[0]

tensor([36.3672, 47.0000])

In [72]:
copy_matrix = matrix.clone()
for i in range(100):
    max_index = argmax2d(copy_matrix)[0].long()
    print(max_index, copy_matrix[0][0][max_index[0]][max_index[1]])
    copy_matrix[0][0][max_index[0]][max_index[1]] = 0

tensor([36, 47]) tensor(0.7067, dtype=torch.float64)
tensor([35, 47]) tensor(0.6928, dtype=torch.float64)
tensor([115,  37]) tensor(0.6083, dtype=torch.float64)
tensor([115,  36]) tensor(0.6006, dtype=torch.float64)
tensor([36, 48]) tensor(0.5796, dtype=torch.float64)
tensor([35, 48]) tensor(0.5712, dtype=torch.float64)
tensor([36, 46]) tensor(0.5684, dtype=torch.float64)
tensor([114,  37]) tensor(0.5539, dtype=torch.float64)
tensor([114,  36]) tensor(0.5516, dtype=torch.float64)
tensor([35, 46]) tensor(0.5415, dtype=torch.float64)
tensor([37, 47]) tensor(0.4752, dtype=torch.float64)
tensor([116,  37]) tensor(0.4651, dtype=torch.float64)
tensor([74, 32]) tensor(0.4442, dtype=torch.float64)
tensor([116,  36]) tensor(0.4413, dtype=torch.float64)
tensor([34, 47]) tensor(0.4216, dtype=torch.float64)
tensor([73, 32]) tensor(0.4164, dtype=torch.float64)
tensor([115,  38]) tensor(0.3996, dtype=torch.float64)
tensor([74, 33]) tensor(0.3944, dtype=torch.float64)
tensor([115,  35]) tensor(0.3937